# Анализ ДТП в России на основе данных dtp-stat.ru
Это пример простого анализа статистики ДТП в России.
Посмотрим на следующие свойства данных:
- Структурированность данных
- Чистота данных
- Распределения наблюдаемых значений и признаков
- Корреляция по значений по признакам

## Шаг -1. Загрузка данных в локальные файлы
Этот шаг можно пропустить, если вы уже загружали файлы ранее.

In [ ]:
import requests as r
import os
import aiohttp
from bs4 import BeautifulSoup as bs

links = []
resp = r.get('https://dtp-stat.ru/opendata/')
if resp.ok:
    soup = bs(resp.text, "html.parser")
    for link in soup.findAll('a'):
        url = link.get('href')
        if '.geojson' in url: links.append('https://dtp-stat.ru' + url)
if len(links) == 85:
    print('Все регионы присутствуют на сайте. Загружаем')
else:
    print("Some regions are missed")
async with aiohttp.ClientSession() as session:
    for link in links:
        async with session.get(link) as resp:
            file_name = os.path.basename(link)
            with open('data/dtp-stat/' + file_name, 'w') as f:
                f.write(await resp.text())
                print('.', end='')
print(' готово.', end='')

## Шаг 0. Склейка файлов в один большой
Склеиваем все скачанные geojson файлы в один большой. Понадобится *много*(~15gb) памяти.

Этот шаг стоит выполнить если вы собираетесь анализировать весь массив данных сразу.

In [ ]:
import glob

files = glob.glob('data/dtp-stat/*')
import json

features = []
for file in files:
    with open(file) as f:
        d = json.load(f)
        features.extend(d['features'])
result = {
    "type": "FeatureCollection",
    "features": features
}
with open('data/dtp-stat/russia.geojson', 'w') as f:
    json.dump(result, f, ensure_ascii=False)
del result
del features

## Шаг 1.  Готовим pandas фрейм

Загружаем данные geojson файла в pandas фрейм. Посмотрим на структуру 


In [1]:
import pandas as pd
import json
from IPython.display import display, Markdown

FILE_NAME = 'data/dtp-stat/altaiskii-krai.geojson'

with open(FILE_NAME) as f:
    d = map(lambda x: x['properties'], json.load(f)['features'])
df = pd.DataFrame.from_dict(d)
df.head(5)

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories
0,2311491,[Дорожно-транспортные происшествия],"В темное время суток, освещение отсутствует","{'lat': 53.342, 'long': 83.6903}","[Многоквартирные жилые дома, Административные ...",Барнаул,610,"г Барнаул, тракт Павловский, 249 а",[Ясно],Наезд на препятствие,2017-09-04 22:15:00,Легкий,"[{'year': 2002, 'brand': 'TOYOTA', 'color': 'Ч...",0,[],1,Алтайский край,[Сухое],2,[Все участники]
1,2489516,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"{'lat': 53.369248, 'long': 83.699802}","[Многоквартирные жилые дома, Нерегулируемый пе...",Барнаул,300,"г Барнаул, ул Юрина, 241",[Дождь],Столкновение,2020-09-24 09:30:00,Легкий,"[{'year': 2006, 'brand': 'SUBARU', 'color': 'С...",0,[],2,Алтайский край,[Мокрое],3,"[Дети, Все участники]"
2,2565463,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"{'lat': 53.343391, 'long': 83.704684}","[Многоквартирные жилые дома, Школа либо иная д...",Барнаул,740,"г Барнаул, тракт Павловский, 68",[Ясно],Наезд на пешехода,2021-04-20 18:55:00,Легкий,"[{'year': 1988, 'brand': 'TOYOTA', 'color': 'Б...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Алтайский край,"[Сухое, Отсутствие, плохая различимость горизо...",2,"[Дети, Пешеходы, Все участники]"
3,2565477,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 53.380807, 'long': 83.690007}","[Жилые дома индивидуальной застройки, Нерегули...",Барнаул,820,"г Барнаул, ул Совхозная, 10 а",[Пасмурно],Наезд на пешехода,2021-04-28 12:20:00,Легкий,"[{'year': 2011, 'brand': 'FIAT', 'color': 'Бел...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Алтайский край,[Сухое],2,"[Пешеходы, Все участники]"
4,2565484,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 53.351736, 'long': 83.718964}","[Многоквартирные жилые дома, Жилые дома индиви...",Барнаул,770,"г Барнаул, ул Советской Армии, 123",[Дождь],Наезд на пешехода,2021-04-27 21:00:00,Тяжёлый,"[{'year': 2002, 'brand': 'HONDA', 'color': 'Че...",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",1,Алтайский край,"[Мокрое, Отсутствие, плохая различимость гориз...",2,"[Пешеходы, Все участники]"


Предущий шаг загрузил данные в датафрейм в памяти и вывел первые 5 строк фрейма.
С этим уже можно работать, но нужно прибраться в типах данных.
Добавим индекс, дату, списки и строки сделаем категориями.

In [2]:
df.set_index('id', inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'])
for column in ['scheme', 'category', 'severity', 'region', 'parent_region']:
    df[column] = df[column].astype('category')
df['address'] = df['address'].astype('string')

В наборе данных у нас пока присуствуют три основных показателя
 - `participants_count`
 - `injured_count`
 - `dead_count`
Посмотрим на их описание

In [4]:
for column in ['participants_count', 'injured_count', 'dead_count']:
    display(Markdown(df[column].describe().to_markdown()))

|       |   participants_count |
|:------|---------------------:|
| count |          20223       |
| mean  |              2.40533 |
| std   |              1.1026  |
| min   |              1       |
| 25%   |              2       |
| 50%   |              2       |
| 75%   |              3       |
| max   |             38       |

|       |   injured_count |
|:------|----------------:|
| count |    20223        |
| mean  |        1.27671  |
| std   |        0.894425 |
| min   |        0        |
| 25%   |        1        |
| 50%   |        1        |
| 75%   |        1        |
| max   |       34        |

|       |    dead_count |
|:------|--------------:|
| count | 20223         |
| mean  |     0.0891559 |
| std   |     0.348517  |
| min   |     0         |
| 25%   |     0         |
| 50%   |     0         |
| 75%   |     0         |
| max   |     8         |

Следом рассмотрим категорийные признаки

In [5]:
for column in ['scheme', 'category', 'severity', 'region', 'parent_region']:
    display(Markdown(df[column].describe().to_markdown()))
    display(Markdown(df[column].value_counts().to_markdown()))

|        |   scheme |
|:-------|---------:|
| count  |    19516 |
| unique |       60 |
| top    |      740 |
| freq   |     1620 |

|     |   scheme |
|----:|---------:|
| 740 |     1620 |
| 500 |     1473 |
| 610 |     1415 |
| 960 |     1272 |
| 070 |     1271 |
| 820 |     1211 |
| 600 |     1096 |
| 200 |     1059 |
| 300 |      886 |
| 880 |      635 |
| 830 |      573 |
| 840 |      558 |
| 710 |      507 |
| 950 |      489 |
| 430 |      461 |
| 700 |      421 |
| 910 |      367 |
| 130 |      325 |
| 940 |      288 |
| 210 |      287 |
| 220 |      284 |
| 140 |      243 |
| 850 |      196 |
| 930 |      187 |
| 760 |      176 |
| 730 |      169 |
| 720 |      141 |
| 860 |      136 |
| 030 |      136 |
| 800 |      127 |
| 770 |      119 |
| 420 |      113 |
| 330 |      104 |
| 040 |      102 |
| 400 |       98 |
| 870 |       95 |
| 120 |       94 |
| 060 |       90 |
| 810 |       89 |
| 110 |       76 |
| 050 |       71 |
| 920 |       67 |
| 410 |       65 |
| 750 |       57 |
| 780 |       53 |
| 010 |       47 |
| 900 |       36 |
| 100 |       35 |
| 090 |       22 |
| 190 |       17 |
| 980 |       14 |
| 020 |       12 |
| 440 |       10 |
| 340 |        6 |
| 230 |        4 |
| 320 |        3 |
| 970 |        3 |
| 620 |        2 |
| 310 |        2 |
| 630 |        1 |

|        | category          |
|:-------|:------------------|
| count  | 20223             |
| unique | 18                |
| top    | Наезд на пешехода |
| freq   | 7053              |

|                                                                                                                       |   category |
|:----------------------------------------------------------------------------------------------------------------------|-----------:|
| Наезд на пешехода                                                                                                     |       7053 |
| Столкновение                                                                                                          |       6940 |
| Опрокидывание                                                                                                         |       1667 |
| Падение пассажира                                                                                                     |       1288 |
| Съезд с дороги                                                                                                        |       1004 |
| Наезд на препятствие                                                                                                  |        910 |
| Наезд на велосипедиста                                                                                                |        687 |
| Наезд на стоящее ТС                                                                                                   |        497 |
| Иной вид ДТП                                                                                                          |         61 |
| Наезд на животное                                                                                                     |         37 |
| Наезд на гужевой транспорт                                                                                            |         19 |
| Отбрасывание предмета                                                                                                 |         19 |
| Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ                         |         18 |
| Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы                             |         14 |
| Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность             |          4 |
| Падение груза                                                                                                         |          2 |
| Наезд на внезапно возникшее препятствие                                                                               |          2 |
| Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении. |          1 |

|        | severity   |
|:-------|:-----------|
| count  | 20223      |
| unique | 3          |
| top    | Легкий     |
| freq   | 12866      |

|             |   severity |
|:------------|-----------:|
| Легкий      |      12866 |
| Тяжёлый     |       5832 |
| С погибшими |       1525 |

|        | region   |
|:-------|:---------|
| count  | 20223    |
| unique | 68       |
| top    | Барнаул  |
| freq   | 8823     |

|                             |   region |
|:----------------------------|---------:|
| Барнаул                     |     8823 |
| Бийск                       |     1720 |
| Рубцовск                    |      938 |
| Первомайский район          |      731 |
| Новоалтайск                 |      702 |
| Тальменский район           |      373 |
| Павловский район            |      350 |
| Бийский район               |      339 |
| Алейский район              |      333 |
| Славгородский район         |      307 |
| Алтайский район             |      284 |
| Троицкий район              |      233 |
| Заринск                     |      215 |
| Рубцовский район            |      213 |
| Косихинский район           |      207 |
| Зональный район             |      207 |
| Смоленский район            |      204 |
| Благовещенский район        |      195 |
| Шипуновский район           |      188 |
| Каменский район             |      172 |
| Поспелихинский район        |      152 |
| Заринский район             |      132 |
| Шелаболихинский район       |      130 |
| Топчихинский район          |      124 |
| Мамонтовский район          |      121 |
| Целинный район              |      117 |
| Калманский район            |      114 |
| Кулундинский район          |      112 |
| Ребрихинский район          |      106 |
| Михайловский район          |      105 |
| Красногорский район         |      103 |
| Залесовский район           |       96 |
| Яровое                      |       95 |
| Локтевский район            |       92 |
| Егорьевский район           |       87 |
| Немецкий национальный район |       84 |
| Романовский район           |       84 |
| Змеиногорский район         |       79 |
| Советский район             |       74 |
| Краснощековский район       |       71 |
| Волчихинский район          |       68 |
| Ключевский район            |       67 |
| Завьяловский район          |       67 |
| Хабарский район             |       67 |
| Бурлинский район            |       64 |
| Усть-Пристанский район      |       61 |
| Усть-Калманский район       |       59 |
| Солонешенский район         |       58 |
| Быстроистокский район       |       58 |
| Петропавловский район       |       58 |
| Белокуриха                  |       58 |
| Крутихинский район          |       58 |
| Курьинский район            |       56 |
| Родинский район             |       54 |
| Тюменцевский район          |       54 |
| Панкрушихинский район       |       52 |
| Ельцовский район            |       52 |
| Третьсяковский район        |       51 |
| Угловский район             |       48 |
| Тогульский район            |       47 |
| Чарышский район             |       45 |
| Солтонский район            |       35 |
| Новичихинский район         |       35 |
| Баевский район              |       34 |
| Табунский район             |       32 |
| Кытмановский район          |       30 |
| Суетский район              |       25 |
| Сибирский                   |       18 |

|        | parent_region   |
|:-------|:----------------|
| count  | 20223           |
| unique | 1               |
| top    | Алтайский край  |
| freq   | 20223           |

|                |   parent_region |
|:---------------|----------------:|
| Алтайский край |           20223 |

### ...

## Примеры похожих исследований на Kaggle
- [https://www.kaggle.com/sobhanmoosavi/us-accidents/tasks](us-accidents)
- [https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales](1.6 million UK traffic accidents)
- [https://www.kaggle.com/amadeus1996/eda-visualizations-for-accidents-in-france](Visualizations for Accidents in France)